# README.

This notebook is the entrypoint for Azure ML enabled evaluation of trained Neural Networks.
In its essence, it connects to Azure ML, makes sure that everything is ready over there, and starts the evaluation.
To that end, this notebook gathers all necessary sourcecodes in a temp-folder, which will be pushed to Azure ML for evaluation.

# Imports.

In [1]:
from azureml.core import Workspace, Dataset
from azureml.core import Experiment
import os
import shutil

# Create temp folder and copy code.

Here you have to be very precise, which code to copy.
And most importantly, which code NOT to copy.

In [2]:
print("Creating temp folder...")
temp_path = "tmp_evaluate"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

print("Copying files...")
shutil.copy(os.path.join("code", "evaluate.py"), temp_path)
shutil.copy(os.path.join("code", "preprocessing.py"), temp_path)
shutil.copytree(os.path.join("code", "gapnet"), os.path.join(temp_path, "gapnet"))

print("Done.")

Creating temp folder...
Copying files...
Done.


# Connect to azure workspace.

Make sure that you have a config.json file with the keys subscription_id, resource_group, and cgm-ml-dev. Either here (not so nice) or in a parent folder (okay but not perfect), or in the root folder of this repo (way to go).

In [3]:
workspace = Workspace.from_config()
workspace

Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev')

# Get the experiment.

- You should always arrange all your runs in an experiment.
- Create at least one experiment per sprint.
- Make sure that the name of the experiment reflects the sprint number.
- On top of that you could also add other tokens to the name. For example network architecture, dataset name, and/or targets.# Get the experiment.

In [4]:
from azureml.core import Experiment
experiment_name = "s2-gapnet-height"
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
s2-gapnet-height,cgm-ml-dev,Link to Azure Machine Learning studio,Link to Documentation


# Find/create a compute target.

Connects to a compute cluster on Azure ML.
If the compute cluster does not exist, it will be created.

Note: Usually computer clusters autoscale. This means that new nodes are created when necessary. And unused VMs will be shut down.

In [5]:
import os
import urllib
import shutil
import azureml
 
from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "tristan-gpu"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

Found existing compute target.


AmlCompute(workspace=Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev'), name=tristan-gpu, id=/subscriptions/9b82ecea-6780-4b85-8acf-d27d79028f07/resourceGroups/cgm-ml-dev/providers/Microsoft.MachineLearningServices/workspaces/cgm-ml-dev/computes/tristan-gpu, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

# Get the dataset for evaluation.

Here you specify which dataset to use.

Note: Double check on Azure ML that you are using the right one.

Note: This dataset is not necessarily the one you used for training.

In [6]:
#dataset_name = "cgmmldevpremium-SampleDataset-Example" # Tiny dataset.
dataset_name = "2019_11_14-bmz_prod_uncleaned" # Full dataset.
dataset = workspace.datasets[dataset_name]
dataset

{
  "source": [
    "('premiumfileshare', '2019_11_14-bmz_prod_uncleaned/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "25493ad0-44b0-4772-a3d4-bf6bd8d44662",
    "name": "2019_11_14-bmz_prod_uncleaned",
    "version": 1,
    "workspace": "Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev')"
  }
}

# Specify the run that contains the trained model.

Make super sure that you put the right experiment- and run-id here.
Double check with Azure ML.
The evaluation script will grab the model as a binary file from that run and use it for evaluation.

In [7]:
experiment_name = "s1-gapnet-height"
run_id = "s1-gapnet-weight_1579792118_0c294b2b"

# Push the evaluation source code to Azure.

In [8]:
from azureml.train.dnn import TensorFlow
TensorFlow.get_supported_versions()

['1.10', '1.12', '1.13', '2.0']

In [11]:
# Specify pip packages here.
pip_packages = [
    "azureml-dataprep[fuse,pandas]",
    "glob2"
]

# Specify script parameters. Most importantly, the id of the run that contains the model.
script_params = {
    "--experiment_name": experiment_name, # The experiment that contains the model.
    "--run_id": run_id # The run that contains the model.
}

# Create the estimator.
estimator = TensorFlow(
    source_directory=temp_path,
    compute_target=compute_target,
    entry_script="evaluate.py",
    script_params=script_params,
    use_gpu=True,
    framework_version="2.0",
    inputs=[dataset.as_named_input("dataset").as_mount()],
    pip_packages=pip_packages
)

# Set compute target.
estimator.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(estimator)

# Show outpus.
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# Delete temp folder.

After all code has been pushed to Azure ML, the temp folder will be removed.

In [ ]:
shutil.rmtree(temp_path)